In [44]:
import pandas as pd 
import numpy as np 
import distance
import re

In [69]:
df_products = pd.read_csv("HOME_MISSION_SHARE/Recomendations/products_data_fr_fr_2017-11-14_150627.187511.csv", \
                          delimiter=";", encoding="latin1")
df_alerts = pd.read_csv("HOME_MISSION_SHARE/Recomendations/alerts_fr_fr_2017-11-14_151447.572712.csv", \
                        delimiter=";", encoding="latin1")



In [70]:
df_products.head()

,title_fr_fr,id,description_fr_fr
0,HTC Windows 8S 4 Go - Noir - Bouygues,222,"\tSmartphone sous Android 2.1, le WildFire vie..."
1,Nokia Lumia 1020 32 Go - Blanc - SFR,228,Compatible 4G - Windows Phone 8 - Ecran AMOLED...
2,iPhone 4 16 Go - Noir - Débloqué,1178,<p><b>iPhone 4 16 Go - Noir Débloqué</b></p>\...
3,iPhone 4 16 Go - Blanc - Débloqué,1179,<strong> Le mobile sera livré sous le dernier ...
4,iPhone 4 32 Go - Noir - Débloqué,1180,<p><b>iPhone 4 32 Go - Noir - Débloqué</b></p>...


In [71]:
df_alerts.head()

,pk,product,criterias,country_code,creation_date
0,30254,IPAD MINI 4 32 GO WIFI + 4g Or Débloqué,NaN,fr_fr,2017-11-14 13:09:03+00:00
1,30252,EXPRESSO À CAPSULES Dolce Gusto Krups Yy1786fd,NaN,fr_fr,2017-11-14 12:18:30+00:00
2,30244,ACER SWIFT Sf314-51-39zj 14 Core I3-6006u 2 Gh...,NaN,fr_fr,2017-11-14 11:43:47+00:00
3,30226,IPAD 4 128 GO WIFI NOIR,NaN,fr_fr,2017-11-14 07:52:01+00:00
4,30217,COMPTEUR DE FANS Facebook 5 Chiffres,NaN,fr_fr,2017-11-14 06:27:33+00:00


In [72]:

df_alerts["product"] = df_alerts["product"].map(lambda x : x.upper().replace('-','')) \
+ " " + df_alerts["criterias"].fillna("").map(lambda x : str(x.upper().replace('-','')))

df_products.iloc[:,0] = df_products.iloc[:,0].map(lambda x : x.upper().replace('-','')) 

In [73]:
df_alerts

,pk,product,criterias,country_code,creation_date
0,30254,IPAD MINI 4 32 GO WIFI + 4G OR DÉBLOQUÉ,NaN,fr_fr,2017-11-14 13:09:03+00:00
1,30252,EXPRESSO À CAPSULES DOLCE GUSTO KRUPS YY1786FD,NaN,fr_fr,2017-11-14 12:18:30+00:00
2,30244,ACER SWIFT SF3145139ZJ 14 CORE I36006U 2 GHZ ...,NaN,fr_fr,2017-11-14 11:43:47+00:00
3,30226,IPAD 4 128 GO WIFI NOIR,NaN,fr_fr,2017-11-14 07:52:01+00:00
4,30217,COMPTEUR DE FANS FACEBOOK 5 CHIFFRES,NaN,fr_fr,2017-11-14 06:27:33+00:00
5,30216,ASUS ZENFONE 2 SELFIE 32 GO BLEU DÉBLOQUÉ,NaN,fr_fr,2017-11-13 23:34:03+00:00
6,30207,DORO 8031 8 GO NOIR DÉBLOQUÉ,NaN,fr_fr,2017-11-13 22:02:28+00:00
7,30199,VENTILATEUR DYSON COOL AM07,NaN,fr_fr,2017-11-13 20:39:24+00:00
8,30198,LG G6 32 GO NOIR DÉBLOQUÉ,NaN,fr_fr,2017-11-13 20:42:47+00:00
9,30197,"IPAD MINI 4 7,9'' 32 GO WIFI ARGENT",NaN,fr_fr,2017-11-13 20:32:07+00:00


In [74]:
df_products.head()

def words(x, word):
    try : 
        word = word[:word.find(re.search(r'\d', word).group(0))]+ " "+word[word.find(re.search(r'\d', word).group(0)):]
    except:
        pass
    
    #Le premier mot est toujours le plus important dans les alertes
    if word.split()[0] in x:
        x = x[x.find(word.split()[0]):]
        #print(x, word)
        return distance.levenshtein(x[:len(word)], word)
    else :
        return 999
prod = "IPHONE7"
df_products["leveinshtein"] = df_products["title_fr_fr"].map(lambda x : words(x, prod))
print(df_products[df_products["leveinshtein"] == df_products["leveinshtein"].min() ].iloc[0,0])


IPHONE 7 128 GO  NOIR DE JAIS  DÉBLOQUÉ


In [75]:
re.search(r'\d', word).group(0)

'5'

In [76]:
prod_res = []
for prod in df_alerts["product"]:
    df_products["leveinshtein"] = df_products["title_fr_fr"].map(lambda x : words(x, prod))
    print(df_products[df_products["leveinshtein"] == df_products["leveinshtein"].min() ].iloc[0,0])
    

IPAD MINI 4 32 GO WIFI + 4G OR  DÉBLOQUÉ
EXPRESSO À CAPSULES DOLCE GUSTO KRUPS YY1786FD
ACER SWIFT SF3145139ZJ 14 CORE I36006U 2 GHZ   SSD 128 GO  RAM 4 GO
IPAD 4 16 GO  WIFI  NOIR  BOUYGUES
COMPTEUR DE FANS FACEBOOK 5 CHIFFRES
ASUS ZENFONE 2 SELFIE 32 GO  BLEU  DÉBLOQUÉ
DORO 8031 8 GO  NOIR  DÉBLOQUÉ
VENTILATEUR DYSON COOL AM07
LG G6 32 GO  NOIR  DÉBLOQUÉ
IPAD MINI 4  7,9'' 32 GO  WIFI  ARGENT
ROBOT PÂTISSIER KITCHENAID 5KSM150PSECR
ECOUTEURS SAMSUNG GEAR ICONX 4 GO  NOIR
EXPRESSO À CAPSULES DOLCE GUSTO KRUPS YY1786FD
SAMSUNG GALAXY A5 (2017) 32 GO  NOIR  DÉBLOQUÉ
SONY XPERIA Z5 PREMIUM 32 GO  NOIR  DÉBLOQUÉ
IPHONE 6S PLUS 128 GO  GRIS SIDÉRAL  DÉBLOQUÉ
ROBOT MULTIFONCTIONS RUSSELL HOBBS RH19006
IPHONE 7 128 GO  NOIR DE JAIS  DÉBLOQUÉ
ASUS NEXUS 7  7 32 GO    NOIR
MACBOOK PRO RETINA 13 CORE I5 2 GHZ   SSD 256 GO  RAM 8 GO


In [77]:
df_alerts

,pk,product,criterias,country_code,creation_date
0,30254,IPAD MINI 4 32 GO WIFI + 4G OR DÉBLOQUÉ,NaN,fr_fr,2017-11-14 13:09:03+00:00
1,30252,EXPRESSO À CAPSULES DOLCE GUSTO KRUPS YY1786FD,NaN,fr_fr,2017-11-14 12:18:30+00:00
2,30244,ACER SWIFT SF3145139ZJ 14 CORE I36006U 2 GHZ ...,NaN,fr_fr,2017-11-14 11:43:47+00:00
3,30226,IPAD 4 128 GO WIFI NOIR,NaN,fr_fr,2017-11-14 07:52:01+00:00
4,30217,COMPTEUR DE FANS FACEBOOK 5 CHIFFRES,NaN,fr_fr,2017-11-14 06:27:33+00:00
5,30216,ASUS ZENFONE 2 SELFIE 32 GO BLEU DÉBLOQUÉ,NaN,fr_fr,2017-11-13 23:34:03+00:00
6,30207,DORO 8031 8 GO NOIR DÉBLOQUÉ,NaN,fr_fr,2017-11-13 22:02:28+00:00
7,30199,VENTILATEUR DYSON COOL AM07,NaN,fr_fr,2017-11-13 20:39:24+00:00
8,30198,LG G6 32 GO NOIR DÉBLOQUÉ,NaN,fr_fr,2017-11-13 20:42:47+00:00
9,30197,"IPAD MINI 4 7,9'' 32 GO WIFI ARGENT",NaN,fr_fr,2017-11-13 20:32:07+00:00


In [28]:
df_alerts.head()

,pk,product,criterias,country_code,creation_date
0,30254,IPAD MINI 4 32 GO WIFI + 4g Or Débloqué,NaN,fr_fr,2017-11-14 13:09:03+00:00
1,30252,EXPRESSO À CAPSULES Dolce Gusto Krups Yy1786fd,NaN,fr_fr,2017-11-14 12:18:30+00:00
2,30244,ACER SWIFT Sf314-51-39zj 14 Core I3-6006u 2 Gh...,NaN,fr_fr,2017-11-14 11:43:47+00:00
3,30226,IPAD 4 128 GO WIFI NOIR,NaN,fr_fr,2017-11-14 07:52:01+00:00
4,30217,COMPTEUR DE FANS Facebook 5 Chiffres,NaN,fr_fr,2017-11-14 06:27:33+00:00


In [16]:
score = {}
for product in df_products["title_fr_fr"]: 
    
for item in df_alerts["product"][0].split(" "): 
    

['IPAD', 'MINI', '4', '32', 'GO', 'WIFI', '+', '4g', 'Or', 'Débloqué']